In [3]:
import os
from supabase import create_client, Client

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

The goal is to be able to
1. Store conversation and message history for each user in the database
2. Store the rating and feedback for each conversation
3. Be able to update the rating and feedback for each conversation
4. Be able to query the conversation and message history for a particular user



In [ ]:
create_table_query = """
CREATE TABLE conversations (
    conversation_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id UUID NOT NULL,
    is_guest BOOLEAN DEFAULT FALSE,
    created_at TIMESTAMP DEFAULT NOW(),
    updated_at TIMESTAMP DEFAULT NOW(),
    rating INTEGER CHECK (rating BETWEEN 1 AND 5),
    feedback TEXT
);

CREATE TABLE messages (
    message_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    conversation_id UUID REFERENCES conversations(conversation_id) ON DELETE CASCADE,
    sender VARCHAR(10) CHECK (sender IN ('user', 'bot')),
    text TEXT NOT NULL,
    timestamp TIMESTAMP DEFAULT NOW()
);
"""

In [12]:
# Bulk Insertion of Messages
def bulk_insert_messages(messages: list[dict]) -> dict:
    try:
        response = supabase.table("messages").insert(messages).execute()
        return response
    except Exception as exception:
        return exception
    
# Bulk Insertion of Conversations
def bulk_insert_conversations(conversations: list[dict]) -> dict:
    try:
        response = supabase.table("conversations").insert(conversations).execute()
        return response
    except Exception as exception:
        return exception

In [40]:
sample_messages = [
    {
        "conversation_id": "123e4567-e89b-12d3-a456-426614174000",
        "sender": "user",
        "text": "How do I implement a binary search tree in Python?",
        "timestamp": "2024-01-15T10:30:00Z"
    },
    {
        "conversation_id": "123e4567-e89b-12d3-a456-426614174000", 
        "sender": "bot",
        "text": "I'll help you implement a binary search tree in Python. First, let's create a Node class to represent each node in the tree...",
        "timestamp": "2024-01-15T10:30:05Z"
    },
    {
        "conversation_id": "123e4567-e89b-12d3-a456-426614174001",
        "sender": "user",
        "text": "What are the best practices for error handling in FastAPI?",
        "timestamp": "2024-01-15T11:15:00Z"
    },
    {
        "conversation_id": "123e4567-e89b-12d3-a456-426614174001",
        "sender": "bot",
        "text": "For error handling in FastAPI, there are several best practices you should follow: 1. Use HTTPException for API errors...",
        "timestamp": "2024-01-15T11:15:10Z"
    }
]

sample_conversations = [
    {
        "conversation_id": "123e4567-e89b-12d3-a456-426614174000",
        "user_id": "a0eebc999c0b4ef8bb6d6bb9bd380a11",
        "created_at": "2024-01-15T10:45:00Z",
        "updated_at": "2024-01-15T10:45:00Z",
        "is_guest": False,
        "rating": 4,
        "feedback": "Great conversation!"
    },
    {
        "conversation_id": "123e4567-e89b-12d3-a456-426614174001",
        "user_id": "a0eebc999c0b4ef8bb6d6bb9bd380a11",
        "created_at": "2024-01-15T10:30:00Z",
        "updated_at": "2024-01-15T10:30:00Z",
        "is_guest": True,
        "rating": 5,
    }
]


In [41]:
bulk_insert_conversations(sample_conversations)

APIResponse[~_ReturnT](data=[{'conversation_id': '123e4567-e89b-12d3-a456-426614174000', 'user_id': 'a0eebc99-9c0b-4ef8-bb6d-6bb9bd380a11', 'is_guest': False, 'created_at': '2024-01-15T10:45:00', 'updated_at': '2024-01-15T10:45:00', 'rating': 4, 'feedback': 'Great conversation!'}, {'conversation_id': '123e4567-e89b-12d3-a456-426614174001', 'user_id': 'a0eebc99-9c0b-4ef8-bb6d-6bb9bd380a11', 'is_guest': True, 'created_at': '2024-01-15T10:30:00', 'updated_at': '2024-01-15T10:30:00', 'rating': 5, 'feedback': None}], count=None)

In [42]:
bulk_insert_messages(sample_messages)

APIResponse[~_ReturnT](data=[{'message_id': 'f439799b-1930-403a-baa6-8b68d1d60d56', 'conversation_id': '123e4567-e89b-12d3-a456-426614174000', 'sender': 'user', 'text': 'How do I implement a binary search tree in Python?', 'timestamp': '2024-01-15T10:30:00'}, {'message_id': '7b665f97-c4d2-4e13-a480-7ede03d3a3a0', 'conversation_id': '123e4567-e89b-12d3-a456-426614174000', 'sender': 'bot', 'text': "I'll help you implement a binary search tree in Python. First, let's create a Node class to represent each node in the tree...", 'timestamp': '2024-01-15T10:30:05'}, {'message_id': 'c63f0944-837e-49e8-91ab-61d4ce1fc92d', 'conversation_id': '123e4567-e89b-12d3-a456-426614174001', 'sender': 'user', 'text': 'What are the best practices for error handling in FastAPI?', 'timestamp': '2024-01-15T11:15:00'}, {'message_id': 'f5969332-80d2-4b93-b3b7-c6a8532d1d36', 'conversation_id': '123e4567-e89b-12d3-a456-426614174001', 'sender': 'bot', 'text': 'For error handling in FastAPI, there are several best p

In [43]:
def get_user_conversations(user_id: str) -> dict:
    """
    Retrieve all conversations for a specific user. Sorted by created_at in descending order.
    
    Args:
        user_id (str): The UUID of the user
        
    Returns:
        dict: API response containing the conversations
    """
    try:
        response = (
            supabase.table("conversations")
            .select("*")
            .eq("user_id", user_id)
            .order("created_at", desc=True)
            .execute()
        )
        return response
    except Exception as exception:
        return exception

def get_conversation_messages(conversation_id: str) -> dict:
    """
    Retrieve all messages for a specific conversation.
    
    Args:
        conversation_id (str): The UUID of the conversation
        
    Returns:
        dict: API response containing the messages
    """
    try:
        response = (
            supabase.table("messages")
            .select("*")
            .eq("conversation_id", conversation_id)
            .execute()
        )
        return response
    except Exception as exception:
        return exception

# Example usage:
user_convos = get_user_conversations("a0eebc999c0b4ef8bb6d6bb9bd380a11")
convo_messages = get_conversation_messages("123e4567-e89b-12d3-a456-426614174000")

In [44]:
user_convos.data

[{'conversation_id': '123e4567-e89b-12d3-a456-426614174000',
  'user_id': 'a0eebc99-9c0b-4ef8-bb6d-6bb9bd380a11',
  'is_guest': False,
  'created_at': '2024-01-15T10:45:00',
  'updated_at': '2024-01-15T10:45:00',
  'rating': 4,
  'feedback': 'Great conversation!'},
 {'conversation_id': '123e4567-e89b-12d3-a456-426614174001',
  'user_id': 'a0eebc99-9c0b-4ef8-bb6d-6bb9bd380a11',
  'is_guest': True,
  'created_at': '2024-01-15T10:30:00',
  'updated_at': '2024-01-15T10:30:00',
  'rating': 5,
  'feedback': None}]

In [45]:
convo_messages.data

[{'message_id': 'f439799b-1930-403a-baa6-8b68d1d60d56',
  'conversation_id': '123e4567-e89b-12d3-a456-426614174000',
  'sender': 'user',
  'text': 'How do I implement a binary search tree in Python?',
  'timestamp': '2024-01-15T10:30:00'},
 {'message_id': '7b665f97-c4d2-4e13-a480-7ede03d3a3a0',
  'conversation_id': '123e4567-e89b-12d3-a456-426614174000',
  'sender': 'bot',
  'text': "I'll help you implement a binary search tree in Python. First, let's create a Node class to represent each node in the tree...",
  'timestamp': '2024-01-15T10:30:05'}]

In [47]:
def update_conversation_feedback(conversation_id: str, feedback: str) -> dict:
    """
    Update the feedback for a specific conversation.
    
    Args:
        conversation_id (str): The UUID of the conversation
        feedback (str): The feedback text to update
        
    Returns:
        dict: API response after updating the feedback
    """
    try:
        response = (
            supabase.table("conversations")
            .update({"feedback": feedback})
            .eq("conversation_id", conversation_id)
            .execute()
        )
        return response
    except Exception as exception:
        return exception

In [48]:
update_conversation_feedback("123e4567-e89b-12d3-a456-426614174001", "Bot does not perform well!")

APIResponse[~_ReturnT](data=[{'conversation_id': '123e4567-e89b-12d3-a456-426614174001', 'user_id': 'a0eebc99-9c0b-4ef8-bb6d-6bb9bd380a11', 'is_guest': True, 'created_at': '2024-01-15T10:30:00', 'updated_at': '2024-01-15T10:30:00', 'rating': 5, 'feedback': 'Bot does not perform well!'}], count=None)

In [50]:
def update_conversation_rating(conversation_id: str, rating: str) -> dict:
    """
    Update the feedback for a specific conversation.
    
    Args:
        conversation_id (str): The UUID of the conversation
        feedback (int): 1 to 5 star ratings for the conversation
        
    Returns:
        dict: API response after updating the feedback
    """
    try:
        response = (
            supabase.table("conversations")
            .update({"rating": rating})
            .eq("conversation_id", conversation_id)
            .execute()
        )
        return response
    except Exception as exception:
        return exception

In [51]:
update_conversation_rating("123e4567-e89b-12d3-a456-426614174001", 1)

APIResponse[~_ReturnT](data=[{'conversation_id': '123e4567-e89b-12d3-a456-426614174001', 'user_id': 'a0eebc99-9c0b-4ef8-bb6d-6bb9bd380a11', 'is_guest': True, 'created_at': '2024-01-15T10:30:00', 'updated_at': '2024-01-15T10:30:00', 'rating': 1, 'feedback': 'Bot does not perform well!'}], count=None)

In [52]:
def delete_conversation(conversation_id: str) -> dict:
    """
    Delete a specific conversation.
    
    Args:
        conversation_id (str): The UUID of the conversation
    
    Returns:
        dict: API response after deleting the conversation
    """
    try:            
        response = (
            supabase.table("conversations")
            .delete()
            .eq("conversation_id", conversation_id)
            .execute()
        )
        return response
    except Exception as exception:
        return exception

In [53]:
delete_conversation("123e4567-e89b-12d3-a456-426614174001")

APIResponse[~_ReturnT](data=[{'conversation_id': '123e4567-e89b-12d3-a456-426614174001', 'user_id': 'a0eebc99-9c0b-4ef8-bb6d-6bb9bd380a11', 'is_guest': True, 'created_at': '2024-01-15T10:30:00', 'updated_at': '2024-01-15T10:30:00', 'rating': 1, 'feedback': 'Bot does not perform well!'}], count=None)